In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler(feature_range=(0,1))
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense,SimpleRNN

In [16]:
ticker = "aapl"
months = 1
# aapl, msft, sber, mtss, moes, rosn, pfe, ko, intc, ma, ibm, baba, five
df=pd.read_csv("data/" + ticker.upper() + "_5Y.csv")

            Date        Open        High         Low       Close   Adj Close  \
1253  2021-05-24  116.669998  118.580002  114.699997  115.320000  115.320000   
1254  2021-05-25  115.349998  120.019997  115.349998  119.040001  119.040001   
1255  2021-05-26  119.620003  121.599998  118.209999  120.040001  120.040001   
1256  2021-05-27  120.000000  121.970001  118.519997  121.500000  121.500000   
1257  2021-05-28  122.300003  125.870003  118.010002  118.110001  118.110001   

      Volume  
1253  432400  
1254  485600  
1255  367800  
1256  432400  
1257  413100  


In [19]:
MONTH_WORK_DAYS = 21
days_watching = 6*MONTH_WORK_DAYS
# if (months == 6):
#     days_watching = 9*MONTH_WORK_DAYS
    
predict_days = months*MONTH_WORK_DAYS
test_threshold = df.shape[0]

88.300003 85.709999 80.690002 88.970001 94.309998 103.639999 97.809998 97.809998 95.07 95.0 96.32 87.559998 93.690002 95.940002 98.889999 98.730003 101.699997 109.830002 107.870003 109.139999 112.940002 123.709999 125.07 134.600006 138.089996 134.190002 139.699997 139.0 139.240005 138.990005 141.190002 142.309998 153.479996 158.600006 208.949997 182.520004 178.600006 185.729996 197.270004 182.800003 186.050003 186.789993 180.699997 173.490005 160.800003 157.490005 167.220001 175.0 174.919998 168.600006 159.880005 156.889999 153.639999 156.320007 155.800003 166.759995 158.369995 171.800003 203.440002 207.240005 207.169998 208.039993 210.380005 209.869995 207.270004 200.5 207.880005 221.0 210.429993 223.740005 220.759995 227.039993 235.570007 224.979996 237.479996 244.960007 246.639999 234.509995 239.820007 246.179993 242.729996 267.0 261.390015 258.480011 263.75 257.23999 255.509995 259.679993 254.949997 259.470001 270.5 282.470001 293.820007 298.559998 301.959991 285.350006 272.76001 2

In [12]:
data = df.sort_index(ascending=True,axis=0)
df_new = pd.DataFrame(index=range(0,len(df)),columns=['Date','Close'])
for i in range(0,len(data)):
    df_new["Date"][i] = data['Date'][i]
    df_new["Close"][i] = data["Close"][i]
    
df_new.index = df_new.Date
df_new.drop("Date", axis=1, inplace=True)

final_dataset = df_new.values

train_data = final_dataset[0:test_threshold,:]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(final_dataset)

x_train_data,y_train_data = [],[]

for i in range(days_watching,len(train_data)-predict_days):
    x_train_data.append(scaled_data[i-days_watching:i,0])
    y_train_data.append(scaled_data[i+predict_days,0])
x_train_data,y_train_data = np.array(x_train_data),np.array(y_train_data)

x_train_data = np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))

In [13]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50,return_sequences=False, input_shape=(x_train_data.shape[1],1)))
# lstm_model.add(LSTM(units=50,return_sequences=False, input_shape=(x_train_data.shape[1],1)))
# lstm_model.add(LSTM(units=128, return_sequences=True))
# lstm_model.add(LSTM(units=64, return_sequences=False))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mse',optimizer='adam')
# lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=30,batch_size=40,verbose=2, shuffle=False)
lstm_model.save("compiled_models/" + ticker + "_" + str(months) + "m.h5")

Epoch 1/30
26/26 - 1s - loss: 0.0486
Epoch 2/30
26/26 - 1s - loss: 0.0456
Epoch 3/30
26/26 - 1s - loss: 0.0398
Epoch 4/30
26/26 - 1s - loss: 0.0399
Epoch 5/30
26/26 - 1s - loss: 0.0390
Epoch 6/30
26/26 - 1s - loss: 0.0388
Epoch 7/30
26/26 - 1s - loss: 0.0383
Epoch 8/30
26/26 - 1s - loss: 0.0380
Epoch 9/30
26/26 - 1s - loss: 0.0377
Epoch 10/30
26/26 - 1s - loss: 0.0374
Epoch 11/30
26/26 - 1s - loss: 0.0371
Epoch 12/30
26/26 - 1s - loss: 0.0368
Epoch 13/30
26/26 - 1s - loss: 0.0365
Epoch 14/30
26/26 - 1s - loss: 0.0361
Epoch 15/30
26/26 - 1s - loss: 0.0357
Epoch 16/30
26/26 - 1s - loss: 0.0355
Epoch 17/30
26/26 - 1s - loss: 0.0359
Epoch 18/30
26/26 - 1s - loss: 0.0362
Epoch 19/30
26/26 - 1s - loss: 0.0332
Epoch 20/30
26/26 - 1s - loss: 0.0452
Epoch 21/30
26/26 - 1s - loss: 0.0408
Epoch 22/30
26/26 - 1s - loss: 0.0353
Epoch 23/30
26/26 - 1s - loss: 0.0340
Epoch 24/30
26/26 - 1s - loss: 0.0343
Epoch 25/30
26/26 - 1s - loss: 0.0338
Epoch 26/30
26/26 - 1s - loss: 0.0333
Epoch 27/30
26/26 - 1